#This notebook was used to create dataset for non-locust coordinates. We used 4 layers from Nasa Worldview database 
###1:SMAP_L4_Analyzed_Surface_Soil_Moisture
###2:ASTER_GDEM_Color_Index (Elevation)
###3:SMAP_L4_Soil_Temperature_Layer_1 (Soil Temp)
###4 MODIS_Terra_NDVI_8Day

We generate random latitude and longitude in the the North America region with same dates as for Locust co-ordinates in DatasetGeneratorForLocustCoordinates file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rasterio

In [ ]:
import numpy as np
import requests
import os
import pandas as pd
import csv

from io import BytesIO
from matplotlib.pyplot import imshow
from PIL import Image

import rasterio
import rasterio.features
import rasterio.warp
from rasterio import MemoryFile

In [ ]:
CSV =  "/content/drive/My Drive/Endurance - NASA Space Apps 2020/Data/New stuff/New_Locust_Data_1.csv"

Random_CSV = "/content/drive/My Drive/Endurance - NASA Space Apps 2020/Data/New stuff/Random_Lats_Longs_1 - Random_Lats_Longs_1.csv"

List_URL = [
"https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME={}&BBOX={}&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,SMAP_L4_Analyzed_Surface_Soil_Moisture&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134",
"https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME={}&BBOX={}&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,ASTER_GDEM_Color_Index&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134",
"https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME={}&BBOX={}&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,SMAP_L4_Soil_Temperature_Layer_1&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134",
"https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME={}&BBOX={}&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,MODIS_Terra_NDVI_8Day&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134"
]

#1:Soil_Moisture 
#2:ASTER_GDEM_Color_Index (Elevation)
#3:SMAP_L4_Soil_Temperature_Layer_1 (Soil Temp)
#4 MODIS_Terra_NDVI_8Day




unit_length = .07
                  
#df = pd.read_csv(CSV)

df2 = pd.read_csv(Random_CSV)

time_add = "T00:00:00Z"

In [ ]:
feature_list = ['Soil_Moisture', 'Elevation', 'Soil_Temperature', 'Vegetation_Index']

In [ ]:
def get_val(lat, longo, time, ix, ID=None):

  lower_lat = lat - unit_length
  left_long = longo - unit_length

  upper_lat = lat + unit_length
  right_long = longo + unit_length 

  Bbox = (lower_lat,left_long,upper_lat,right_long)

  Bbox = str(Bbox)[1:]

  Bbox = Bbox[:-1]

  Bbox = Bbox.replace(" ", "")

  for i in range(len(List_URL)):
      
      #print(str(ID-22258) + ':' + str(i))

      link = List_URL[i]

      url = link.format(time, Bbox)
      response = requests.get(url)
      #print(url)

      img = BytesIO(response.content)

      #Image.open(img)

      with open("example2.tif", "wb") as  f:
        #print("here")
        f.write(img.getvalue())

      with rasterio.open('example2.tif') as dataset:
        # Read the dataset's valid data mask as a ndarray.
        mask = dataset.dataset_mask()
        #print(type(mask))
        # Extract feature shapes and values from the array.
        for geom, val in rasterio.features.shapes(
                mask, transform=dataset.transform):
            #print(geom ,val, dataset.crs)
            # Transform shapes from the dataset's own coordinate
            # reference system to CRS84 (EPSG:4326).
            geom = rasterio.warp.transform_geom(
              dataset.crs, 'EPSG:4326', geom, precision=6)

            # Print GeoJSON shapes to stdout.
            #print("Here",geom, dataset)
            b = dataset.index(longo, lat)
            a = dataset.read()
            #print(a.shape) # (3,850,800)
            index = a[:,b[0]-1,b[1]]

            index = '#%02x%02x%02x' % (index[0], index[1],index[2])

            df2.loc[ix, feature_list[i]] = index
            #df.loc[ix, feature_list[i]] = index
            #print(index)

            
            




In [ ]:
import random

def random_coord():
  min_lat = 25
  max_lat = 48

  min_long = -120
  max_long = -66
  return (random.uniform(min_lat, max_lat), random.uniform(min_long, max_long))



In [ ]:
for ix in df2.index:
  print(ix)
  row=df2.loc[ix]
  time=row.Time
  time = (time + 'T00:00:00Z')
  lat,longo = random_coord()
  df2.loc[ix, 'Latitude '] = lat 
  df2.loc[ix, 'Longitude '] = longo
  #print(df2.head())
  #print(lat,longo)
  get_val(lat, longo, time, ix)

df2.to_csv('Updated_CSV_Random_2.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
df2.to_csv('Updated_CSV_Random_1.csv')

In [ ]:
df2.head()

,Latitude,Longitude,ID,Time,Locust,Soil_Moisture,Elevation,Soil_Temperature,Vegetation_Index
0,-1,-1,22258,2020-06-30,0,-1,-1,-1,-1
1,-1,-1,22259,2020-06-30,0,-1,-1,-1,-1
2,-1,-1,22260,2020-06-30,0,-1,-1,-1,-1
3,-1,-1,22261,2020-06-30,0,-1,-1,-1,-1
4,-1,-1,22262,2020-06-30,0,-1,-1,-1,-1
